In [1]:
from sqlalchemy import create_engine
from operator import itemgetter
import json
import math
import time

import pandas as pd
import numpy as np
#import seaborn as sns
from sklearn import linear_model
from skopt import gp_minimize
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=6, suppress=True)

In [6]:
race_type = 'R'
# category = 'G'
# category = 'H'

bet_type = 'Q'

In [11]:
engine = create_engine('sqlite:///../../data/exotic.db')
conn = engine.connect()

dfo = pd.read_sql_table('exotic', conn)
print('{} exotics loaded!'.format(len(dfo)))

307571 exotics loaded!


In [12]:
df = dfo.loc[dfo['race_type'].isin([race_type])]
print('{} {} races for {} betting!'.format(len(df), race_type, bet_type))

307571 R races for Q betting!


In [13]:
df.tail(10)

,id,race_type,bet_type,res1,res2,res3,res4,num_runners,run1_num,run1_win_perc,...,run4_num,run4_win_perc,run4_win_scaled,run4_win_rank,run4_place_perc,run4_place_scaled,run4_place_rank,prediction,success,dividend
307561,307562,R,Q,6,5,11,9,13.0,11,0.038168,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000842,0,89.1
307562,307563,R,Q,6,5,11,9,13.0,11,0.038168,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000829,0,89.1
307563,307564,R,Q,6,5,11,9,13.0,11,0.038168,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000600,0,89.1
307564,307565,R,Q,6,5,11,9,13.0,11,0.038168,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000494,0,89.1
307565,307566,R,Q,6,5,11,9,13.0,5,0.032154,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000741,0,89.1
307566,307567,R,Q,6,5,11,9,13.0,5,0.032154,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000522,0,89.1
307567,307568,R,Q,6,5,11,9,13.0,5,0.032154,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000497,0,89.1
307568,307569,R,Q,6,5,11,9,13.0,9,0.032154,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000530,0,89.1
307569,307570,R,Q,6,5,11,9,13.0,9,0.032154,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000511,0,89.1
307570,307571,R,Q,6,5,11,9,13.0,13,0.012048,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000475,0,89.1


In [14]:
# bet size
balance = 1000
bet_chunk = balance * 0.01

In [16]:
def process_strategy(dfp, strategy, x):
    """run strategy, must pass runners returned back into results (it might be modified)"""
    book = []
    for _, comb in df.iterrows():
        try:
            comb = strategy(comb, bet_chunk, bet_type, race_type, x)
        except Exception as e:
            print(comb)
            raise
            
#         if num_bets:
#             bet_results(book, runners, bet_chunk, num_bets)
            
        break
            
    book_df = pd.DataFrame(book)
    return book_df

In [17]:
def bet_results(book, runners, bet_chunk, num_bets):
    """get results from bets"""
    place_cutoff = 3 if len(runners) >= 8 else 2

    win_diff = 0
    max_diff = 0
    prob2scales = []
    prob0scales = []
    # profit start with loss of bet made
    outcome = {
        'success': 0,
        'profit': -bet_chunk,
        'num_bets': num_bets,
        'num_runners': len(runners),
    }
    for i, runner in enumerate(runners):
        diff = abs(runner['win_scaled'] - runner[prob])
        max_diff = max(max_diff, diff)
        if int(runner['finishingPosition']) == 1:
            win_diff = diff

        if runner[bet]:
            prob2scales.append(runner[prob] / runner['win_scaled'])
            prob0scales.append(runner[prob] - runner['win_scaled'])

            if bet_type == 'W':
                if int(runner['finishingPosition']) == 1:
                    outcome['success'] = 1
                    # profit will be payout added against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['win_odds']
                    outcome['profit'] += winnings
                break
            elif bet_type == 'P':
                if 1 <= int(runner['finishingPosition']) <= place_cutoff:
                    # success is how correct each placement was
                    outcome['success'] += 1 / num_bets
                    # profit will be return added and offset against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['place_odds']
                    outcome['profit'] += winnings

    outcome['max_diff'] = max_diff
    outcome['win_diff'] = win_diff
    outcome['prob2scale_avg'] = sum(prob2scales) / len(prob2scales)
    outcome['prob2scale'] = min(prob2scales)
    outcome['prob0scale'] = min(prob0scales)
    outcome['bet_chunk'] = bet_chunk
#     outcome['runners'] = runners
    
    # add odds curve
#     probs = sorted([r['win_odds'] for r in runners if runner['win_odds']])
#     p = np.polyfit(np.arange(len(probs)), probs, 2)
#     outcome['pa'] = p[0]
#     outcome['pb'] = p[1]
#     outcome['pc'] = p[2]
    
    book.append(outcome)

In [ ]:
def bet_on_pred(comb, bet_chunk, bet_type, race_type, x):
    """bet on prediction"""

    comb['bet'] = 0
    if comb['prediction'] > x[0]:
        comb['bet'] = bet_chunk

In [ ]:
def f(x):
    book_df = process_strategy(dfo, bet_positive_dutch, x)
    #print(book_df)
    if not len(book_df):
        return 0
    profit = book_df.profit.sum()
#     if bet_type == 'W':
#         participation = (len(book_df) ** 2) / (len(dfo) ** 2)
#     else:
#         participation = 1
#     return profit * participation * -1
    return profit / len(dfo) * -1
    

print('optimizing...')
time_start = time.time()

# x0 => min profit
# x1 => min probs2scale
# fail => avg profit

x, *grid = sp.optimize.brute(f, np.s_[0:2:4j, 0.9:1.4:16j], full_output=True)

print('done: took {:.0f}s'.format((time.time() - time_start)))
# print('x: {}'.format(x))
# print('grid: {}'.format(grid))

In [ ]:
print('category: {}'.format(category))
print('bet type: {}'.format(bet_type))
print('x: {}'.format(x))
df = process_strategy(dfo, bet_positive_dutch, x)
print('${:.2f} profit per race'.format(df.profit.sum() / len(dfo)))
print('{:.0f}% of races {} / {}'.format(len(df) / len(dfo) * 100, len(df), len(dfo)))

In [ ]:
df['ones'] = np.ones(len(df))
df['win_rate'] = df.success.cumsum() / df.ones.cumsum()
win_title = '{:.0f}% win rate'.format(df.win_rate.iloc[-1] * 100)
df.win_rate.plot(title=win_title, xlim=(100), ylim=(0, 1))

In [ ]:
# ROI
roi = df.profit.rolling(20).sum() / df.bet_chunk.rolling(20).sum()
roi.plot(xlim=(100), ylim=(-1, 1), alpha=0.3)
roi = df.profit.cumsum() / df.bet_chunk.cumsum()
roi_title = '{:.1f}% ROI'.format(roi.iloc[-1] * 100)
roi.plot(xlim=(100), ylim=(-1, 1), title=roi_title)

In [ ]:
# profit per num bets 
df.num_bets.plot(kind='hist', title='# bets')
groups = df.groupby('num_bets').mean()
groups.plot(kind='bar', y='success', title='Win rate per #bets')
groups.plot(kind='bar', y='profit', title='Profit per #bets')

In [ ]:
# profit per num runners
df.num_runners.hist()
groups = df.groupby('num_runners').mean()
groups.plot(kind='bar', y='success', title='Win rate per #runners')
groups.plot(kind='bar', y='profit', title='Profit per #runners')

In [ ]:
# max diff
df.win_diff.plot(kind='hist', bins=200, alpha=0.6, ylim=(0, 100), xlim=(0, 0.3))
df.max_diff.plot(kind='hist', bins=200, alpha=0.2, ylim=(0, 100), xlim=(0, 0.3))

In [ ]:
# df.profit.plot(kind='hist', bins=50, xlim=(-5), ylim=(0, 140))
df.profit.rolling(100).sum().plot(alpha=0.4)
df.profit.cumsum().plot(alpha=0.8)

In [ ]:
# bets/runners
df['bets_vs_runners'] = df.num_bets / df.num_runners
df['bets_vs_runners'].hist(bins=10)
groups = df.groupby(df['bets_vs_runners'].apply(lambda x: round(x, 3))).sum()
groups.plot(kind='line', y='profit', title='mean profit/bets_per_runners')

In [ ]:
# # pred vs scale
# df['prob_vs_scale'] = df.pr / df.num_runners
# df['bets_vs_runners'].hist(bins=10)
# groups = df.groupby(df['bets_vs_runners'].apply(lambda x: round(x, 3))).sum()
# groups.plot(kind='line', y='profit', title='mean profit/bets_per_runners')

In [ ]:
df.prob2scale.plot(kind='hist', bins=500, title='min prob2scale', xlim=(1, 1.5))
groups = df.groupby(df['prob2scale'].apply(lambda x: round(x, 1))).mean()
groups.plot(kind='bar', y='profit', title='profit / prob2scale')
groups.plot(kind='bar', y='win_rate', title='winrate / prob2scale')

groups_sum = df.groupby(df['prob2scale'].apply(lambda x: round(x, 1))).sum()
groups_sum.plot(kind='bar', y='profit', title='sum profit / prob2scale')

In [ ]:
df.prob0scale.plot(kind='hist', bins=50, title='min prob0scale')
groups = df.groupby(df['prob0scale'].apply(lambda x: round(x, 2))).mean()
groups.plot(kind='bar', y='profit', title='profit / prob0scale')
groups.plot(kind='bar', y='win_rate', title='winrate / prob0scale')

groups_sum = df.groupby(df['prob0scale'].apply(lambda x: round(x, 2))).sum()
groups_sum.plot(kind='bar', y='profit', title='total profit')